In [ ]:
import pandas as pd
import re

## Inside Airbnb Data

In [ ]:
air_df = pd.read_csv("../data/research/air-bnb-listings.csv", sep=';')
air_df['City'] = air_df['City'].str.replace('-', ' ')
# Convert to uppercase for later matching
air_df['City'] = air_df['City'].str.upper()
# Strip 'City' from City names (e.g. NEW YORK not NEW YORK CITY)
# But also converts JERSEY CITY to JERSEY
air_df['City'] = air_df['City'].str.replace(' CITY', '')
air_df['City'] = air_df['City'].str.replace(' COUNTY', '')
air_df.head()

#### Global

In [ ]:
airbnb_city = pd.Series(air_df['City'].unique())
len(airbnb_city)

In [ ]:
airbnb_countries = pd.Series(air_df['Country'].unique())
len(airbnb_countries)

#### United States

Notice that dataset still contains 'NV', 'OR, 'MSA', 'DC'

In [ ]:
air_us_cities = air_df[air_df['Country'].str.contains('United states')]
airbnb_us_cities = pd.Series(sorted(air_us_cities['City'].unique()))
airbnb_us_cities

## Kaggle Cost of Living, etc. for Global Cities

In [ ]:
costs_df = pd.read_csv("../data/Cost_of_living_index.csv")
costs_df.rename(columns = {'City':'City_Country'}, inplace = True)
costs_df

In [ ]:
# Split City and Country
cc_df = costs_df['City_Country'].apply(lambda x: pd.Series(x.split(',', 1)))

# Remove State Abbreviations from US entries
cc_df[1] = cc_df[1].apply(lambda x: pd.Series(re.sub(r'\w*,\s', '',x)))

# Clean up Dataframe
city_costs_df = pd.concat([cc_df, costs_df], axis=1)
city_costs_df.rename(columns={0:'City',1:'Country'},inplace=True)

# Convert to uppercase for later matching
city_costs_df['City'] = city_costs_df['City'].str.upper()

city_costs_df['City'] = city_costs_df['City'].str.replace(' CITY', '')
city_costs_df['City'] = city_costs_df['City'].str.replace(' COUNTY', '')

#### Number of Cities

In [ ]:
city_costs = pd.Series(city_costs_df['City'].unique())
len(city_costs)

#### Number of Countries

In [ ]:
country_costs = pd.Series(city_costs_df['Country'].unique())
len(country_costs)

## Kaggle Population for Global Cities

In [ ]:
pop_df = pd.read_csv('../data/cities15000.csv', encoding = "ISO-8859-1")
# Convert to uppercase for later matching
pop_df['asciiname'] = pop_df['asciiname'].str.upper()

pop_df['asciiname'] = pop_df['asciiname'].str.replace(' CITY', '')
pop_df['asciiname'] = pop_df['asciiname'].str.replace(' COUNTY', '')

pop_df.head()

#### Total Cities

In [ ]:
city_pop = pd.Series(pop_df['asciiname'].unique())
len(city_pop)

#### Total Country Codes

In [ ]:
country_pop = pd.Series(pop_df['country code'].unique())
len(country_pop)

## Find Common Cities

### Cities in both population and Cost of X Sets

In [ ]:
pop_costs = pd.Series(list(set(city_costs) & set(city_pop)))
pop_costs

### Cities in Cost Set but not Population - (Spelling differences, etc.)

In [ ]:
cost_cities = pd.Series(list(set(city_costs) - set(city_pop)))
cost_cities

### Cities in Airbnb Dataset and Cost of X Sets

In [ ]:
air_cost_list = pd.Series(list(set(airbnb_city) & set(city_costs)))
len(air_cost_list)

### Cities and Airbnb, Population, AND Cost of X Sets

#### What cities can't we find a match for in the common dataset (of Pop and Cost of X)
* Edinburgh is spelled Edinburg
* Hawaii is a state, Barwon South West Vic - region/provinence
* Visualisations?

Gives us an idea of extent of data cleaning we would need to automate it


In [ ]:
# Airbnb minus the intersection of found cities
missing_cities = (set(airbnb_city)) - (set(airbnb_city) & set(pop_costs))
missing_cities                        

#### Cities Found in Airbnb AND Population AND Cost of X

In [ ]:
air_pop_cost_list = sorted((list(set(airbnb_city) & set(pop_costs))))
print(len(air_pop_cost_list))
print(air_pop_cost_list)

#### Total listings for All Cities in Airbnb, Population, and Cost of X Sets

In [ ]:
global_cities = air_df.groupby("City", as_index=False)["Room ID"].count()
global_city_listings = global_cities[global_cities['City'].isin(air_pop_cost_list)==True]
pd.Series(global_city_listings['Room ID']).sum()

#### US Cities in Airbnb and Population and Cost of X Sets

In [ ]:
us_air_pop_cost = pd.Series(list(set(airbnb_us_cities) & set(pop_costs))).tolist()
print(len(us_air_pop_cost))
print(us_air_pop_cost)

#### Total Listings for US Cities in Airbnb, Population, and Cost of X Sets

In [ ]:
cities = air_df.groupby("City", as_index=False)["Room ID"].count()
city_listings = cities[cities['City'].isin(us_air_pop_cost)==True]
pd.Series(city_listings['Room ID']).sum()

#### City Explorations

In [ ]:
amst_df = pd.read_csv("../data/listings/Amsterdam_08June2020.csv")
amst_df

In [ ]:
amst_df.room_type.unique()

In [ ]:
city_costs_df

In [ ]:
pop_df

In [ ]:
x = pop_df['admin1 code'].unique()
